In [1]:
import requests
import uuid
import json
import time
import os
import random

# ПОЛУЧАЕМ ДАННЫЕ О СПИСКЕ КОЛЛЕКЦИЙ

In [35]:
addr = "http://cluster.vstu.ru:57888"
#addr = "http://localhost:57888"
id = str(uuid.uuid4())
message = { "id": id, "mqtt_key": "234", "user": "2321", "type": "courses", "data_key": "","action": "get_data" }
resp = requests.post(f"{addr}/api/get_courses_data", json=message)
resp.json()

{'data': {'courses': ['test']}}

In [30]:
message = { "id": id }
resp = requests.post("http://localhost:57888/api/get_message_result", json=message)
resp.json()

{'error': 'Message ID not found in results'}

# ПОЛУЧАЕМ ДАННЫЕ О СТРУКТУРЕ КОЛЛЕКЦИИ (АСИНХРОННЫЙ СПОСОБ)

In [11]:
id = str(uuid.uuid4())
message = { "id": id, "mqtt_key": "234", "user": "2321", "type": "problems", "data_key": "test", "action": "get_data" }
resp = requests.post("http://localhost:57888/api/add_message", json=message)
resp

<Response [200]>

In [12]:
message = { "id": id }
resp = requests.post(
    "http://localhost:57888/api/get_message_result", json=message)
resp


<Response [200]>

In [13]:
resp.json()

{'message': {'id': '9ca156d9-10d0-4dce-8f44-649a549ba0b7',
  'mqtt_key': '234',
  'user': '2321',
  'type': 'problems',
  'data_key': 'test',
  'action': 'get_data',
  'timestamp': 1671697929},
 'result': {'problems': [{'1': ['1'],
    'rating': 1,
    'task': '1. Напишите программу для знакомства с человеком и приветствия его (пользователь вводит Имя и Фамилию в двух отдельных строках, программа выводит в новой строке приветствие, например, при вводе имени «Иван» и фамилии «Петров» программа выводит : «Здравствуйте, Иван Петров !»)'},
   {'2': ['1'],
    'rating': 3,
    'task': '2. Напишите программу, которая выбирает максимальное и минимальное из пяти введенных чисел.'},
   {'3': ['1'],
    'rating': 2,
    'task': '3. Напишите программу, которая находит среднее из пяти введенных чисел.'},
   {'4': ['1'],
    'rating': 2,
    'task': '4. Найти периметр и площадь прямоугольного треугольника, если даны длины его катетов a и b. Значения а и b вводятся с клавиатуры.'},
   {'5': ['1'],
 

# ПОЛУЧАЕМ ДАННЫЕ О СТРУКТУРЕ КОЛЛЕКЦИИ (СИНХРОННЫЙ СПОСОБ)

In [25]:
addr = "http://cluster.vstu.ru:57888"
#addr = "http://localhost:57888"
id = str(uuid.uuid4())
message = {"id": id, "mqtt_key": "234", "user": "2321",
           "type": "problems", "data_key": "test", "action": "get_data"}
resp = requests.post(f"{addr}/api/get_courses_data", json=message)
resp

<Response [200]>

In [26]:
resp.json()

{'data': {'problems': []}}

# УДАЛЕНИЕ КОЛЛЕКЦИИ

In [23]:
addr = "http://cluster.vstu.ru:57888"
#addr = "http://localhost:57888"
course = "test"
id = str(uuid.uuid4())
message = {"id": id, "mqtt_key": "234", "user": "2321",
           "type": "course", "data_key": course, "action": "clear_data"}
resp = requests.post(f"{addr}/api/clear_data", json=message)
resp

<Response [200]>

In [24]:
resp.json()

{'data': {'test': 'Droped!'}}

# СОСТАВЛЕНИЕ ФАЙЛА С ЗАДАЧАМИ

In [6]:
path = "/Users/user/Temp/vstu-pycourse-tests"
tests = []
for number in range(1, 90):
    fn = f'{path}/tests/{number}/result.json'
    fn1 = f'{path}/tasks/{number}.py'
    if os.path.exists(fn):
        with open(fn, encoding='utf-8') as f:
            var = json.load(f)
        with open(fn1, encoding='utf-8') as f:
            txt = f.read()
            ind1 = txt.find("'''")
            ind2 = txt.find("'''", 10)
            txt = txt[ind1+3:ind2].replace("\n", " ")
        test = {"problem": number, "rating": random.choice([1,2,3]), "task": txt, "type": "equal", "variants": {"1": var}}
        tests.append(test)
with open(f"{path}/test_problems.json", "w", encoding='utf-8') as f:
    json.dump(tests, f, ensure_ascii=False)

# ДОБАВЛЕНИЕ ЗАДАЧ ИЗ ФАЙЛА

In [44]:
course = "test"
addr = "http://cluster.vstu.ru:57888"
#addr = "http://localhost:57888"
fn = "tests2.json"
with open(fn, encoding='utf-8') as f:
    data = json.load(f)
for p in data:
    for v in p["variants"]:
        print(f'Problem {p["problem"]}, variant {v}')
        id = str(uuid.uuid4())
        message = {
            "id": id, 
            "mqtt_key": 123,
            "user": "admin", 
            "type": p["type"],
            "rating": p["rating"],
            "course": course, 
            "problem": p["problem"], 
            "variant": v,
            "action": "add_problem",
            "task": p.get("task", ""),
            "tests": p["variants"][v]
        }
        r = requests.post(f"{addr}/api/add_message", json=message)
        print(f"{r}")
        message = { "id": id }
        while True:
            time.sleep(1)
            resp = requests.post(f"{addr}/api/get_message_result", json=message)
            if resp.status_code == 200 and 'message' in resp.text:
                break
                print(f"{resp.json()}")


Problem 104, variant 1
<Response [200]>
Problem 104, variant 2
<Response [200]>
Problem 104, variant 3
<Response [200]>


# ПОЛУЧАЕМ ДАМП БАЗЫ

In [43]:
message = { "date": "2023-01-02", "processor_name": "equal_processor", "admin_key": "e86e915a-249f-4fdc-a48a-7c3f9a0b36ff" }
resp = requests.post(
    "http://cluster.vstu.ru:57888/api/get_base_dump", json=message)
resp

KeyboardInterrupt: 

In [41]:
resp.json()["data"][-1]

{'message': {'id': 'ecc69169-6674-49f6-8515-88c7cfde594a',
  'mqtt_key': '123',
  'user': 'ОленниковаАА_СУБД6',
  'language': 'python3',
  'course': 'test',
  'action': 'test_problem',
  'problem': 104,
  'variant': '1',
  'code': 's = input()\ne = "!()-вЂ”[]{};:,.<>/\\\\?@#$%^&*_~"\nso = ""\nfor c in s:\n    if c not in e:\n        so += c\nso = so.replace(\'  \', \' \')\nprint(so + "\\n")\n       \n            \n        ',
  'timestamp': 1672680568},
 'result': {'1': {'score': 0,
   'test_out': 'Это ёж — его голыми руками не озьмёшь',
   'test_in': 'Это ёж — его голыми руками не возьмёшь!'},
  'success_count': 0,
  'res_score': 0,
  'max_res_score': 10,
  'timestamp': 1672680568}}

In [63]:
resp.json()

{'error': 'Need admin_key for process request!'}

# ДОБАВЛЯЕМ/ИЗМЕНЯЕМ ИНФОРМАЦИЮ ПОЛЬЗОВАТЕЛЯ

In [3]:
message = { "user_name": "123456", "data": {"var": [1,2,3]} }
resp = requests.post(
    "http://localhost:57888/api/add_user_info", json=message)
resp

<Response [200]>

In [35]:
resp.json()

{'data': 'sucess'}

# ПОЛУЧАЕМ ИНФОРМАЦИЮ ПОЛЬЗОВАТЕЛЯ

In [38]:
message = { "user_name": "ОленниковаАА_СУБД6" }
resp = requests.post(
    "http://cluster.vstu.ru:57888/api/get_user_info", json=message)
resp

<Response [200]>

In [39]:
resp.json()

{'data': {'user_name': 'ОленниковаАА_СУБД6',
  'data': {'test': [{'11': ['1', '2', '3', '4', '5'],
     'rating': 2,
     'task': 'Напишите программу, которая выводит на экран квадрат из звездочек состороной N.\nНапример, на входе ожидается: \n2\nНа выходе ожидается:\n**\n**',
     'last_result': 'Задача проверена.\nРезультат:\nНабрано 10 баллов из 10 возможных.\n'},
    {'35': ['1', '2', '3', '4', '5'],
     'rating': 2,
     'task': 'Напишите программу, которая вводит с клавиатуры натуральное число N и определяет его факториал, т.е. произведение натуральных чисел от 1 до N. Что будет, если ввести большое значение N (например, N = 20)?\nНапример, на входе: \n3\nНа выходе \n6',
     'last_result': 'Задача проверена.\nРезультат:\nНабрано 10 баллов из 10 возможных.\n'},
    {'49': ['1', '2', '3', '4', '5'],
     'rating': 3,
     'task': 'Совершенным называется число, равное сумме всех своих делителей, меньших его самого (например, 6 = 1+2+3). Напишите программу, которая вводит натуральн